<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-24 13:17:43
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -58.01 K (-0.41%)
Current PnL: -23.15 L (-15.17%)
CY Booked + Current PnL: -9.26 L (-6.07%)
-------------------
Total profit:  1.54 L
Total loss:  -24.70 L
-------------------
Total Booked + Current PnL: 17.72 L (14.1%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.74%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.57 L (62.09%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.24%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.24,-14.03,16.43,0.09,14234.0,-14140.0,86637.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.82,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.25,-8.76,9.63,0.02,15337.0,-15300.0,159259.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
43,ITC,452.00,-0.75,-1.09,11.66,10.44,23081.0,-2191.0,197947.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.41,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.22,6.68,12.67,20.19,23562.0,11643.0,185963.0,-16.49,50.0,X-MC,7.64,68.0,0.49,1.30,22.80,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,-0.34,12.52,7.89,21.40,23564.0,33228.0,298654.0,4.42,59.0,X-LC,7.93,12.0,1.41,2.09,19.99,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,2.83,-13.80,66.26,43.32,93029.0,-22472.0,140400.0,-25.97,41.0,X-SC,1.93,90.0,-0.24,0.98,7.80,X40N,ATH,ELECTRICAL
70,SURYODAY,216.00,2.15,-17.93,49.92,23.03,73363.0,-32109.0,146962.0,60.09,52.0,H-SC,11.23,167.0,-0.44,1.03,45.82,XR,NTT,BANKS
56,QUESS,424.00,1.87,-27.98,98.34,42.84,46039.0,-18190.0,46816.0,-52.62,40.0,X-SC,37.22,83.0,-0.40,0.33,1.87,XY24,NTT,MISC
29,HAPPSTMNDS,1480.71,1.73,-36.89,192.92,84.85,156034.0,-47285.0,80880.0,-25.05,48.0,H-SC,14.25,147.0,-0.30,0.57,3.81,AR,ATH,IT
26,GILLETTE,11206.78,1.59,-0.65,33.28,32.42,83950.0,-1641.0,252255.0,-18.36,36.0,X-SC,6.64,85.0,-0.02,1.77,14.35,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.00,-3.95,-46.38,224.81,74.15,197203.0,-75885.0,87720.0,-257.83,25.0,H-SC,14.60,149.0,-0.38,0.62,12.44,XR,NTT,CHEMICALS
27,GLAXO,3466.20,-3.52,-3.25,43.83,39.15,84504.0,-6484.0,192800.0,-22.38,26.0,X-MC,9.12,60.0,-0.08,1.35,24.80,X40,ATH,PHARMA
72,TANLA,1943.92,-3.19,-36.80,247.63,119.69,423732.0,-99646.0,171115.0,-36.34,26.0,H-SC,12.01,156.0,-0.24,1.20,35.18,AR,ATH,IT
6,ASIANTILES,137.00,-3.10,-15.46,114.90,81.67,91121.0,-14505.0,79305.0,7127.78,44.0,L-SC,18.43,271.0,-0.16,0.56,53.58,XR,NTT,CERAMICS
47,KANSAINER,340.00,-2.91,-22.30,46.05,13.47,96484.0,-60147.0,209520.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.47,6.53,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.18,-0.55,107.87,106.72,132306.0,-683.0,122653.0,-50.82,33.0,H-SC,4.07,139.0,-0.01,0.86,22.69,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.46,-5.38,119.74,107.91,167274.0,-7946.0,139698.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.25,-8.76,9.63,0.02,15337.0,-15300.0,159259.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.24,-14.03,16.43,0.09,14234.0,-14140.0,86637.0,101.07,50.0,M-SC,4.26,253.0,-0.99,0.61,13.82,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.91,-22.30,46.05,13.47,96484.0,-60147.0,209520.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.47,6.53,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.70,-22.78,70.98,32.03,72259.0,-30033.0,101802.0,-16.37,49.0,H-SC,6.62,148.0,-0.42,0.71,14.03,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,1.35,1.64,69.49,72.27,119676.0,2773.0,172221.0,-12.89,58.0,M-LC,4.6,99.0,0.02,1.21,7.91,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.35,1.64,69.49,72.27,119676.0,2773.0,172221.0,-12.89,58.0,M-LC,4.60,99.0,0.02,1.21,7.91,XR,NTT,IT
38,INDIAMART,4810.62,-1.18,-0.55,107.87,106.72,132306.0,-683.0,122653.0,-50.82,33.0,H-SC,4.07,139.0,-0.01,0.86,22.69,AR,ATH,MISC
25,FINCABLES,1641.55,-1.46,-5.38,119.74,107.91,167274.0,-7946.0,139698.0,-22.98,32.0,M-SC,10.37,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.26,-2.28,38.79,35.63,79442.0,-4773.0,204801.0,-15.85,33.0,H-MC,4.52,119.0,-0.06,1.44,14.01,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,0.06,-5.35,92.21,81.92,100188.0,-6146.0,108652.0,-44.55,32.0,M-SC,5.32,236.0,-0.06,0.76,28.63,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,JSWINFRA,342.0,-1.02,-9.99,28.02,15.23,50527.0,-20021.0,180326.0,-22.98,23.0,X-MC,6.51,66.0,-0.40,1.26,19.03,X40N,NTT,REALTY
76,TMPV,600.0,-0.69,-14.75,66.78,42.18,157358.0,-40774.0,235636.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.0,0.45,-35.18,108.58,35.20,90563.0,-45263.0,83407.0,9.31,25.0,X-SC,12.62,92.0,-0.50,0.58,0.45,X40,NTT,FOOTWEAR
27,GLAXO,3466.2,-3.52,-3.25,43.83,39.15,84504.0,-6484.0,192800.0,-22.38,26.0,X-MC,9.12,60.0,-0.08,1.35,24.80,X40,ATH,PHARMA
60,RELAXO,1176.0,-0.88,-47.57,190.48,52.31,144978.0,-69048.0,76112.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.88,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.69,-14.75,66.78,42.18,157358.0,-40774.0,235636.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.75,-1.09,11.66,10.44,23081.0,-2191.0,197947.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.41,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.26,1.54,28.98,30.97,52008.0,2724.0,179462.0,-22.29,50.0,X-MC,1.54,75.0,0.05,1.26,14.30,X40,ATH,INSURANCE
3,ACC,3906.0,-0.37,-23.37,114.23,64.15,208276.0,-55621.0,182330.0,-55.00,42.0,X-SC,1.90,82.0,-0.27,1.28,2.28,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.08,-14.67,47.09,25.51,74779.0,-27295.0,158800.0,-2.45,54.0,H-LC,1.93,49.0,-0.37,1.11,17.02,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.69,-14.75,66.78,42.18,157358.0,-40774.0,235636.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,0.45,-35.18,108.58,35.20,90563.0,-45263.0,83407.0,9.31,25.0,X-SC,12.62,92.0,-0.50,0.58,0.45,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.50,-13.25,71.13,48.45,123722.0,-26568.0,173938.0,-19.44,34.0,X-MC,7.45,64.0,-0.21,1.22,0.50,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.33,-4.33,39.82,33.76,76502.0,-8700.0,192120.0,-32.51,34.0,X-MC,3.21,57.0,-0.11,1.35,0.64,X40,ATH,FMCG
54,PAGEIND,51605.07,-0.13,-5.13,32.88,26.07,51076.0,-8400.0,155340.0,-30.18,35.0,X-MC,8.87,55.0,-0.16,1.09,0.70,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.87,-27.98,98.34,42.84,46039.0,-18190.0,46816.0,-52.62,40.0,X-SC,37.22,83.0,-0.40,0.33,1.87,XY24,NTT,MISC
60,RELAXO,1176.00,-0.88,-47.57,190.48,52.31,144978.0,-69048.0,76112.0,-44.15,28.0,X-SC,3.79,91.0,-0.48,0.53,1.88,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.45,-35.18,108.58,35.20,90563.0,-45263.0,83407.0,9.31,25.0,X-SC,12.62,92.0,-0.50,0.58,0.45,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.28,2.73,20.78,24.08,27100.0,3462.0,130412.0,-5.65,44.0,X-SC,4.74,89.0,0.13,0.91,22.88,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-1.08,-4.93,27.88,21.58,37146.0,-6903.0,133236.0,-52.09,33.0,X-MC,7.02,56.0,-0.19,0.93,17.96,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.63,-13.86,38.47,19.28,114644.0,-47948.0,298008.0,-24.92,66.0,X-LC,7.62,1.0,-0.42,2.09,10.17,X40,ATH,IT
41,INFY,2275.00,1.13,6.37,45.60,54.87,151762.0,19924.0,332812.0,-17.09,63.0,X-LC,3.15,2.0,0.13,2.33,15.18,X40,BTT,IT
76,TMPV,600.00,-0.69,-14.75,66.78,42.18,157358.0,-40774.0,235636.0,-23.08,23.0,X-LC,0.40,3.0,-0.26,1.65,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.21,-9.77,50.36,35.67,143523.0,-30847.0,284995.0,-20.46,40.0,X-LC,6.05,4.0,-0.21,2.00,2.73,X40N,ATH,FMCG
43,ITC,452.00,-0.75,-1.09,11.66,10.44,23081.0,-2191.0,197947.0,-40.74,45.0,X-LC,0.77,5.0,-0.09,1.39,4.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-3.10,-15.46,114.90,81.67,91121.0,-14505.0,79305.0,7127.78,44.0,L-SC,18.43,271.0,-0.16,0.56,53.58,XR,NTT,CERAMICS
68,SONACOMS,806.63,1.39,-11.75,58.12,39.54,51887.0,-11883.0,89276.0,-31.03,69.0,M-SC,6.85,220.0,-0.23,0.63,25.89,AR,ATH,AUTO
51,MASFIN,398.61,-2.08,-4.94,28.40,22.05,26450.0,-4845.0,93135.0,-18.25,51.0,H-SC,7.21,164.0,-0.18,0.65,35.38,XR,ATH,FINANCE
70,SURYODAY,216.00,2.15,-17.93,49.92,23.03,73363.0,-32109.0,146962.0,60.09,52.0,H-SC,11.23,167.0,-0.44,1.03,45.82,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.25,-8.76,9.63,0.02,15337.0,-15300.0,159259.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.25,-8.76,9.63,0.02,15337.0,-15300.0,159259.0,142.75,71.0,M-SC,5.46,234.0,-1.00,1.12,38.53,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.59,0.55,20.29,20.95,51362.0,1373.0,253141.0,-0.49,76.0,X-LC,12.12,31.0,0.03,1.77,36.38,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.05,-37.47,112.53,32.89,54325.0,-28930.0,48276.0,-696.43,64.0,L-MC,5.61,259.0,-0.53,0.34,32.95,XR,NTT,BANKS
0,5PAISA,593.00,-1.91,-35.97,76.33,12.90,106786.0,-78595.0,139901.0,105.53,56.0,H-SC,14.56,161.0,-0.74,0.98,16.33,OX40N,NTT,FINANCE
50,LTIM,7201.88,0.33,8.69,21.13,31.66,56533.0,21397.0,267548.0,4.56,63.0,H-LC,12.50,43.0,0.38,1.88,48.45,X200,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.83
1,25,44.73
2,50,75.51


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.55
MC    29.26
LC    25.20
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.89
X40      21.22
X40N     11.88
XR        9.65
XY25      9.09
AR        8.97
OX40N     7.68
X200      1.88
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.83
X-LC    20.03
M-SC    12.04
X-SC     8.21
H-MC     4.72
H-LC     2.99
L-SC     1.38
M-MC     1.37
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.80,-5.04,39.01
IT,13.86,-14.37,73.41
FINANCE,9.42,-15.32,64.05
MISC,7.43,-22.74,73.02
PAINTS,6.01,-7.70,23.70
ELECTRICAL,5.91,-11.22,51.96
INSURANCE,4.41,-0.49,35.90
PHARMA,3.83,-2.36,35.13
AUTO,3.36,-19.92,71.33


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3059071.0,21
AR,1294958.0,10
XR,1287156.0,13
X40,1010965.0,14
X40N,753352.0,9
OX40N,695630.0,10
XY25,344429.0,6
SR,280489.0,2
MH,73922.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3567970.0,25
M-SC,1343798.0,15
X-MC,1224586.0,16
X-LC,905069.0,11
X-SC,747578.0,8
H-MC,407014.0,3
L-SC,263962.0,3
H-LC,131312.0,2
M-LC,119676.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1222271.0      6
           AR         880281.0      5
           XR         788190.0      7
M-SC       XY24       786735.0      6
X-MC       X40        448904.0      7
X-LC       X40        387548.0      5
X-MC       XY24       348123.0      3
H-SC       OX40N      322817.0      4
X-SC       X40N       291650.0      3
M-SC       OX40N      284450.0      5
X-SC       XY24       281415.0      3
H-SC       SR         280489.0      2
X-MC       X40N       247698.0      4
X-LC       XY24       230164.0      2
H-MC       AR         216651.0      2
X-LC       X40N       214004.0      2
H-MC       XY24       190363.0      1
X-MC       XY25       179861.0      2
L-SC       XR         175599.0      2
X-SC       X40        174513.0      2
M-SC       XR         149366.0      2
           AR         123247.0      2
M-LC       XR         119676.0      1
L-SC       OX40N       88363.0      1
H-LC       AR          74779.0      1
H-SC       MH          73922.0      1
X-LC       XY25        73353.0      2
H-LC       X200        56533.0      1
L-MC       XR          54325.0      1
M-MC       XY25        49966.0      1
L-LC       XY25        41249.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
